<H4>Import Libraries</H4>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import random
from scipy import ndimage
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


<H4>Load CIFAR 10 dataset</H4>

In [2]:
nb_classes = 10
 
def load_dataset():
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    print 'X_train shape:', X_train.shape
    print X_train.shape[0], 'train samples'
    print X_test.shape[0], 'test samples'
 
    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
 
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
 
    return X_train, Y_train, X_test, Y_test

X_train, y_train, X_test, y_test = load_dataset()

X_train shape: (50000L, 3L, 32L, 32L)
50000 train samples
10000 test samples


<H4>Create a model with 4 convolutional layers and 2 fully-connected layers</H4>

In [3]:
def create_network(channels, image_rows, image_cols, lr, decay, momentum):
    model = Sequential()
 
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape = (channels, image_rows, image_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Convolution2D(64, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    return model

model = create_network(3, 32, 32, 0.01, 1e-6, 0.9)

<H4>Fit the model with 10% validation split</H4>

In [4]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64, validation_split=0.1, verbose=1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 2274s - loss: 1.9062 - val_loss: 1.7725
Epoch 2/10
45000/45000 [==============================] - 2266s - loss: 1.4687 - val_loss: 1.6683
Epoch 3/10
45000/45000 [==============================] - 2254s - loss: 1.2919 - val_loss: 1.1747
Epoch 4/10
45000/45000 [==============================] - 2235s - loss: 1.1648 - val_loss: 1.0348
Epoch 5/10
45000/45000 [==============================] - 2227s - loss: 1.0685 - val_loss: 0.9399
Epoch 6/10
45000/45000 [==============================] - 2231s - loss: 0.9905 - val_loss: 1.0355
Epoch 7/10
45000/45000 [==============================] - 2245s - loss: 0.9263 - val_loss: 0.9063
Epoch 8/10
45000/45000 [==============================] - 2235s - loss: 0.8761 - val_loss: 0.7859
Epoch 9/10
45000/45000 [==============================] - 2241s - loss: 0.8350 - val_loss: 0.7754
Epoch 10/10
45000/45000 [==============================] - 2235s - lo

<H4>Verify the accuracy of the model on test set</H4>

In [5]:
test_loss = model.evaluate(X_test, y_test, batch_size = 64, verbose=1)
print 'Test Loss: ', test_loss

10000/10000 [==============================] - 95s    
Test Loss:  0.762864499283


<H4>Save the model in JSON format</H4>

In [9]:
json_model = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(json_model)

<H4>Save the weights in HDF5 format</H4>

In [11]:
model.save_weights('model_weights.h5')